In [39]:
from __future__ import print_function
import argparse
import os
import sys
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
cudnn.fastest = True
import torch.optim as optim
import torchvision.utils as vutils
from torch.autograd import Variable

import models.UNet as net

import models.UNet as net_test
from misc import *
from torchsummary import summary
import datetime
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import torch.nn.functional as F
import torch.optim as optim
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from basic_toolfunc import normalization_test, local_area, coord_trans, number_to_coord
from models.CNN_models import ConvNet_2
from models.gan_models import G, D

from model_test_func import prior_update_gan_noise, CNN_update, CNNGAN_update_path

In [40]:
# load some auxuallary matrices 
c_list_h = np.load('test_code/c_h.npy')
c_list = [number_to_coord(ele) for ele in c_list_h]

b_list_h = np.load('test_code/b_h.npy')
b_list = [number_to_coord(ele) for ele in b_list_h]

a_list_h = np.load('test_code/a_h.npy')
a_list = [number_to_coord(ele) for ele in a_list_h]

c = np.zeros((100,100))
for i in range(100):
    for j in range(100):
        temp = coord_trans(i, j)
        if temp in c_list_h:
            c[i,j] = 1
        if temp in b_list_h:
            c[i,j] = 2
        if temp in a_list_h:
            c[i,j] = 2

channel = c==1
river = c==2
land = c==0

In [41]:
# test of the models across all the test data

# save the result in the following list
result_save = []
# the number of time series, can be chosen from 0,1,2,...,29
i = 10
# load the timeseries data
time_series = np.load('/home/gtx1080/Sync/Kun/30_min_ele/time_series/timeseries_ele_{0}_paper.npy'.format(i))

# functions to choose different trained models to test, here three examples are given
# more trained models can be found in trained models

## Model with mixed discriminator and L1 loss in loss function
# model_no = 5500
# cnnpredict = CNNGAN_update_path(time_series[26,1:4,:,:].reshape((-1,3,100,100)), 
#                              time_series[26, 4:6,:,:].reshape((-1,2,100,100)), 
#                              model_no)

## CNN model
# model_no = 6800
# cnnpredict = CNN_update(time_series[26,1:4,:,:].reshape((-1,3,100,100)), 
#                              time_series[26, 4:6,:,:].reshape((-1,2,100,100)), 
#                              model_no)

# GAN model without posterior adjustment
model_no = 102
cnnpredict = prior_update_gan_noise(time_series[26,1:4,:,:].reshape((-1,3,100,100)), 
                             time_series[26, 4:6,:,:].reshape((-1,2,100,100)), 
                             model_no)


#     cnnpredict, pos_cnnpredict = ensemble_learning_CNN_truestates(time_series[32,1:4,:,:], 
#                                                                   time_series[26,1:4,:,:].reshape((-1,3,100,100)), 
#                                                                   time_series[26, 4:6,:,:].reshape((-1,2,100,100)), 
#                                                                   7800, index_list, 
#                                                                   point_list_2, 
#                                                                   test_cov, obs_var_list)

result_save.append(time_series[26,1:4,:,:].reshape((-1,3,100,100)))
result_save.append(cnnpredict.copy())

# apply it to predict a temporal prediction for 20 steps
for j in range(20):

#     cnnpredict = CNNGAN_update_path(cnnpredict, 
#                                  time_series[32+6*j, 4:6,:,:].reshape((-1,2,100,100)),
#                                  model_no)
#     cnnpredict = CNN_update(cnnpredict, 
#                             time_series[32+6*j, 4:6,:,:].reshape((-1,2,100,100)),
#                             model_no)
    
    cnnpredict = prior_update_gan_noise(cnnpredict, 
                            time_series[32+6*j, 4:6,:,:].reshape((-1,2,100,100)),
                            model_no)

    result_save.append(cnnpredict.copy())
#     result_save = np.array(result_save)
#     np.save('/home/gtx1080/Sync/Kun/paper/output/CNNnoisefree_result_{0}.npy'.format(i), result_save)